В данном файле производится сравнение изображений, сгенерированных с помощью GAN, с изображениями, снятыми на телефон. Изображения не подвергались никакой предварительной обработке кроме уменьшения размера.

In [15]:
import src.Functions as Fu
import src.Filter as Ft
import src.getFingerprint as gF
import src.maindir as md
import src.extraUtils as eu
import numpy as np
import os
import cv2 as cv

import skimage
from skimage import io
from PIL import Image

Для снятия отпечатка камеры была собрана максимально усреднённая выборка из фотографий.

In [2]:
Images = []

for n_pics in range (0, 169):
    img = 'algo_test'+os.sep+'originals'+os.sep+'IMG'+str(n_pics)+'.jpg'
    Images.append(img)
Images

['algo_test\\originals\\IMG0.jpg',
 'algo_test\\originals\\IMG1.jpg',
 'algo_test\\originals\\IMG2.jpg',
 'algo_test\\originals\\IMG3.jpg',
 'algo_test\\originals\\IMG4.jpg',
 'algo_test\\originals\\IMG5.jpg',
 'algo_test\\originals\\IMG6.jpg',
 'algo_test\\originals\\IMG7.jpg',
 'algo_test\\originals\\IMG8.jpg',
 'algo_test\\originals\\IMG9.jpg',
 'algo_test\\originals\\IMG10.jpg',
 'algo_test\\originals\\IMG11.jpg',
 'algo_test\\originals\\IMG12.jpg',
 'algo_test\\originals\\IMG13.jpg',
 'algo_test\\originals\\IMG14.jpg',
 'algo_test\\originals\\IMG15.jpg',
 'algo_test\\originals\\IMG16.jpg',
 'algo_test\\originals\\IMG17.jpg',
 'algo_test\\originals\\IMG18.jpg',
 'algo_test\\originals\\IMG19.jpg',
 'algo_test\\originals\\IMG20.jpg',
 'algo_test\\originals\\IMG21.jpg',
 'algo_test\\originals\\IMG22.jpg',
 'algo_test\\originals\\IMG23.jpg',
 'algo_test\\originals\\IMG24.jpg',
 'algo_test\\originals\\IMG25.jpg',
 'algo_test\\originals\\IMG26.jpg',
 'algo_test\\originals\\IMG27.jpg',
 '

Снятие отпечатка:

In [3]:
RP,_,_ = gF.getFingerprint(Images)
RP = Fu.rgb2gray1(RP)
sigmaRP = np.std(RP)
Fingerprint = Fu.WienerInDFT(RP, sigmaRP)

Снятие отпечатка с изображений, сгенерированных нейросетью, и сравнение с отпечатком камеры.

In [5]:
pce_list = []

for n_pics in range(0,33):
    imx = 'algo_test'+os.sep+'examples'+os.sep+'pic'+str(n_pics)+'.png'
    Noisex = Ft.NoiseExtractFromImage(imx, sigma=2.)
    Noisex = Fu.WienerInDFT(Noisex, np.std(Noisex))
    
    Ix = cv.cvtColor(cv.imread(imx), cv.COLOR_BGR2GRAY)
    C = Fu.crosscorr(Noisex,np.multiply(Ix, Fingerprint))
    
    det, det0 = md.PCE(C)
    pce_list.append(det['PCE'])
    
    
pce_list

[0.7529250113191135,
 0.14297288012697537,
 0.0030366403362533843,
 0.022249559526273336,
 0.2938589037755774,
 0.047867412540900454,
 0.31469973641714793,
 -0.23592898451345876,
 -0.76729786741683,
 3.27250397719999,
 -4.473893302898111,
 6.18923573306114,
 0.44919326598516784,
 0.38891947489125445,
 -0.16212981053790568,
 -0.5079942806584604,
 0.06818457489337508,
 -0.00027117904973290467,
 -0.14857301450035934,
 -0.008026602672028494,
 2.2269945181903616,
 0.10040486343838644,
 0.7045078904473204,
 1.0618248329463347,
 -0.0048218700053540425,
 0.08263414981199418,
 -0.26137238221143516,
 0.2002537502184776,
 -2.0251626813060986,
 -0.2000053058979024,
 -4.354564981011977,
 -0.27341373271756936,
 -0.007804335799141255]

In [24]:
max_value = max(pce_list)

print("Biggest PCE index: ", max_value," Picture Number: ", pce_list.index(max_value))

path = 'algo_test'+os.sep+'examples'+os.sep+'pic'+str(pce_list.index(max_value))+'.png'
img = Image.open(path)
img.show()

Biggest PCE index:  6.18923573306114  Picture Number:  11


Как видно из продемонстрированных данных, большая часть изображений, сгенерированных нейросетью, демонстрирует очень низкий коэффициент подобия (PCE), однако несколько изображений демонстрируют коэффициенты подобия, близкие к приемлемым значениям, самый большой из которых составляет около 6.2.

Для сравнения, изображения из изначальной выборки, демонстрирующие коэффициент подобия >200, после сжатия до размеров 256х256 демонстрируют коэффициент подобия 2.47. 

Данный результат можно считать доказательством того, что подделка отпечатка камеры с помощью генеративных нейросетей (GAN) теоретически возможна.